In [1]:
import pandas as pd
import numpy as np
import os
from abnumber import Chain
from Bio import SeqIO
import requests
from io import StringIO
import time
from tqdm import tqdm
import seaborn as sns
from bin.utils import iterate_fasta
import re
import time

%config InlineBackend.figure_format = 'retina'

In [2]:
SCHEME = 'kabat'
TASK_DIR = '../../data/tasks/therapeutic_rediscovery'

In [3]:
def read_chains(oasis_path):
    vh_table = pd.read_excel(oasis_path, sheet_name='VH', index_col=0)
    vl_table = pd.read_excel(oasis_path, sheet_name='VL', index_col=0)
    scheme = vh_table['scheme'].iloc[0]
    assert scheme == SCHEME, f'Expected OASis in {SCHEME}, got {scheme} in: {oasis_path}'
    vh_chains = Chain.from_dataframe(vh_table, scheme=SCHEME, as_series=True)    
    vl_chains = Chain.from_dataframe(vl_table, scheme=SCHEME, as_series=True)
    return vh_chains, vl_chains

In [4]:
experimental_heavy, experimental_light = read_chains(os.path.join(TASK_DIR, f'thera/humanized_oasis.xlsx'))

In [6]:
genes_heavy = experimental_heavy.apply(lambda chain: chain.find_human_germlines(1)[0][0].name.split('-')[0].split('/')[0].replace('IG',''))
genes_heavy.head()

Abituzumab      HV1
Afutuzumab      HV1
Alacizumab      HV3
Alemtuzumab     HV4
Anrukinzumab    HV3
dtype: object

In [7]:
genes_light = experimental_light.apply(lambda chain: chain.find_human_germlines(1)[0][0].name.split('-')[0].split('/')[0].replace('IG','').replace('D',''))
genes_light.head()

Abituzumab      KV1
Afutuzumab      KV2
Alacizumab      KV1
Alemtuzumab     KV1
Anrukinzumab    KV1
dtype: object

In [8]:
genes_heavy.value_counts()

HV1    74
HV3    62
HV4    13
HV7     8
HV2     4
HV5     3
dtype: int64

In [9]:
genes_light.value_counts()

KV1    104
KV2     21
KV4     16
KV3     15
KV6      4
LV3      1
LV1      1
LV7      1
LV8      1
dtype: int64

## Parental (murine) chains

In [10]:
parental_heavy = {}
parental_light = {}
for record in iterate_fasta(os.path.join(TASK_DIR, f'oas_cdr_hits/parental.fa')):
    if record.description.endswith(' VH'):
        parental_heavy[record.id] = str(record.seq)
    elif record.description.endswith(' VL'):
        parental_light[record.id] = str(record.seq)
    else:
        raise ValueError(record.description)

In [11]:
parental_heavy['Abituzumab']

'QVQLQQSGAELAKPGASVKLSCKASGYTFTSFWMHWVKQRPGQGLEWIGYINPRSGYTEYNEIFRDKATLTADKSSSTAYMQLSSLTSEDSAVYYCARFLGRGAMDYWGQGTSVTVSS'

In [12]:
parental_light['Abituzumab']

'DIQMTQTTSSLSASLGDRVTISCRASQDISNYLAWYQQKPDGTIKLLIYYTSKIHSGVPSRFSGSGSGTDYSLTISNLEQEDIATYFCQQGNTFPYTFGGGTKLEIK'

curl 'http://opig.stats.ox.ac.uk/webapps/newsabdab/sabpred/humab' \
  -H 'Proxy-Connection: keep-alive' \
  -H 'Pragma: no-cache' \
  -H 'Cache-Control: no-cache' \
  -H 'Upgrade-Insecure-Requests: 1' \
  -H 'Origin: http://opig.stats.ox.ac.uk' \
  -H 'Content-Type: multipart/form-data; boundary=----WebKitFormBoundaryd6mMAIYeZfkpIsS8' \
  -H 'User-Agent: Mozilla/5.0 (Macintosh; Intel Mac OS X 10_14_6) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/88.0.4324.150 Safari/537.36' \
  -H 'Accept: text/html,application/xhtml+xml,application/xml;q=0.9,image/avif,image/webp,image/apng,*/*;q=0.8,application/signed-exchange;v=b3;q=0.9' \
  -H 'Referer: http://opig.stats.ox.ac.uk/webapps/newsabdab/sabpred/humab' \
  -H 'Accept-Language: en-GB,en;q=0.9,en-US;q=0.8,cs;q=0.7' \
  -H 'Cookie: _ga=GA1.3.349754799.1573478668; _hjid=ef6d678e-ac4a-4b20-84c0-95957d8d20d8; session=eyJfaWQiOnsiIGIiOiJPR00yT1dSaU5HSXdZamc1TlRVeU1EVTRZekUyWldNelpUQm1PR0poWVRJPSJ9fQ.Ew26Bw.qPsoQNRvmjQFj19rjlX4c6TaAcI; _gid=GA1.3.1842970181.1613945339; _gat_gtag_UA_142143190_1=1' \
  --data-raw $'------WebKitFormBoundaryd6mMAIYeZfkpIsS8\r\nContent-Disposition: form-data; name="h_sequence"\r\n\r\nEVKLLESGGGLVQPGGSMRLSCAGSGFTFTDFYMNWIRQPAGKAPEWLGFIRDKAKGYTTEYNPSVKGRFTISRDNTQNMLYLQMNTLRAEDTATYYCAREGHTAAPFDYWGQGVMVTVSS\r\n------WebKitFormBoundaryd6mMAIYeZfkpIsS8\r\nContent-Disposition: form-data; name="h_v_gene"\r\n\r\nHV1\r\n------WebKitFormBoundaryd6mMAIYeZfkpIsS8\r\nContent-Disposition: form-data; name="h_threshold"\r\n\r\n0.852\r\n------WebKitFormBoundaryd6mMAIYeZfkpIsS8\r\nContent-Disposition: form-data; name="l_sequence"\r\n\r\nDIKMTQSPSFLSASVGDRVTLNCKASQNIDKYLNWYQQKLGESPKLLIYNTNNLQTGIPSRFSGSGSGTDFTLTISSLQPEDVATYFCLQHISRPRTFGTGTKLELK\r\n------WebKitFormBoundaryd6mMAIYeZfkpIsS8\r\nContent-Disposition: form-data; name="l_v_gene"\r\n\r\nKV1\r\n------WebKitFormBoundaryd6mMAIYeZfkpIsS8\r\nContent-Disposition: form-data; name="l_threshold"\r\n\r\n0.851\r\n------WebKitFormBoundaryd6mMAIYeZfkpIsS8\r\nContent-Disposition: form-data; name="jobname"\r\n\r\nTest\r\n------WebKitFormBoundaryd6mMAIYeZfkpIsS8--\r\n' \
  --compressed \
  --insecure

In [13]:
scores = pd.read_csv('../../data/tasks/humanness/thera_sabdab_humab.tsv', sep='\t', index_col=0)
extra = pd.DataFrame([
    ['Inebilizumab', 0.895, 0.655],
    ['Oportuzumab', 0.455, 0.787],
    ['Sacituzumab', 0.45, 0.716],
    ['Sapelizumab', 0.850, 0.748],
    ['Satralizumab', 0.85, 0.748],
    ['Tislelizumab', 0.845, 0.905]
], columns=['Therapeutic','Hu-mAb Score (H)','Hu-mAb Score (L)']).set_index('Therapeutic')
scores = pd.concat([scores[['Hu-mAb Score (H)','Hu-mAb Score (L)']], extra])
scores = scores.loc[parental_heavy.keys()]
scores.head()

,Hu-mAb Score (H),Hu-mAb Score (L)
Therapeutic,,
Abituzumab,0.430,0.619
Afutuzumab,0.830,0.562
Alacizumab,0.935,0.686
Alemtuzumab,0.835,0.842
Anrukinzumab,0.880,0.809


In [14]:
def submit_humanization(h_sequence, h_v_gene, h_threshold, l_sequence, l_v_gene, l_threshold, jobname='Test'):
    data = {
        'h_sequence': h_sequence,
        'h_v_gene': h_v_gene,
        'h_threshold': h_threshold,
        'l_sequence': l_sequence,
        'l_v_gene': l_v_gene,
        'l_threshold': l_threshold,
        'jobname': jobname
    }
    job_request = requests.post('http://opig.stats.ox.ac.uk/webapps/newsabdab/sabpred/humab', data=data)
    jobid = job_request.url.strip('/').split('/')[-1]
    return jobid

def get_humanization_results(jobid, jobname='Test'):
    url = f'http://opig.stats.ox.ac.uk/webapps/newsabdab/sabpred/humab_results/{jobid}'
    heavy_url = f'http://opig.stats.ox.ac.uk/webapps/newsabdab/sabpred/humab_results/{jobid}/{jobname}/heavy'
    light_url = f'http://opig.stats.ox.ac.uk/webapps/newsabdab/sabpred/humab_results/{jobid}/{jobname}/light'
    for wait in range(100):
        result_request = None
        for retry in range(100):
            try:
                result_request = requests.get(url)
                break
            except:
                time.sleep(retry * 5)
        assert result_request is not None, f'Retry failed: {url}'
        time.sleep(wait * 5)
        if 'Error loading Hu-mAb results' in result_request.text or '>failed<' in result_request.text:
            raise ValueError('Hu-mAb error') 
        heavy_request = requests.get(heavy_url)
        if heavy_request.ok:
            heavy_text = heavy_request.text
            light_text = requests.get(light_url).text
            break
    
    regex = re.compile('humanised sequence: (.*)')
    try:
        heavy_seq = regex.search(heavy_text).group(1).strip()
    except:
        print(heavy_text)
        raise ValueError()
    try:
        light_seq = regex.search(light_text).group(1).strip()
    except:
        print(light_text)
        raise ValueError()
    return heavy_seq, light_seq

In [ ]:
# jobid1 = submit_humanization(
#     h_sequence='EVKLLESGGGLVQPGGSMRLSCAGSGFTFTDFYMNWIRQPAGKAPEWLGFIRDKAKGYTTEYNPSVKGRFTISRDNTQNMLYLQMNTLRAEDTATYYCAREGHTAAPFDYWGQGVMVTVSS', 
#     h_v_gene='HV1', 
#     h_threshold=0.01, 
#     l_sequence='DIKMTQSPSFLSASVGDRVTLNCKASQNIDKYLNWYQQKLGESPKLLIYNTNNLQTGIPSRFSGSGSGTDFTLTISSLQPEDVATYFCLQHISRPRTFGTGTKLELK', 
#     l_v_gene='KV1', 
#     l_threshold=0.01, 
#     jobname='Test'
# )

In [25]:
out_dir = os.path.join(TASK_DIR, 'humab')

In [16]:
!mkdir -p {out_dir}

In [ ]:
for name in parental_heavy.keys():
    outpath = os.path.join(out_dir, 'tmp_manual', name+'.fa')
    jobid_path = os.path.join(out_dir, 'tmp_manual', name+'.jobid')
    h_threshold = scores['Hu-mAb Score (H)'][name]
    l_threshold = scores['Hu-mAb Score (L)'][name]
    print(name, h_threshold, l_threshold, outpath)
    if os.path.exists(outpath):
        continue
    jobname = name+'_Rediscovery'
    if os.path.exists(jobid_path):
        with open(jobid_path, 'rt') as f:
            jobid = f.read().strip()
    else:
        jobid = submit_humanization(
            h_sequence=parental_heavy[name], 
            h_v_gene=genes_heavy[name], 
            h_threshold=h_threshold, 
            l_sequence=parental_light[name], 
            l_v_gene=genes_light[name], 
            l_threshold=l_threshold, 
            jobname=jobname
        )
        with open(jobid_path, 'wt') as f:
            f.write(f'{jobid}\n')
    
    print('  Waiting for job:', 'http://opig.stats.ox.ac.uk/webapps/newsabdab/sabpred/humab_results/'+jobid)
    try:
        heavy_seq, light_seq = get_humanization_results(jobid, jobname=jobname)
        with open(outpath, 'wt') as f:
            f.write(f'>{name} VH (threshold {h_threshold})\n')
            f.write(f'{heavy_seq}\n')
            f.write(f'>{name} VL (threshold {l_threshold})\n')
            f.write(f'{light_seq}\n')
    except Exception as e:
        print('  FAILED', str(e))


Abituzumab 0.43 0.619 ../../data/tasks/therapeutic_rediscovery/humab/tmp_manual/Abituzumab.fa
Afutuzumab 0.83 0.562 ../../data/tasks/therapeutic_rediscovery/humab/tmp_manual/Afutuzumab.fa
Alacizumab 0.935 0.6859999999999999 ../../data/tasks/therapeutic_rediscovery/humab/tmp_manual/Alacizumab.fa
Alemtuzumab 0.835 0.8420000000000001 ../../data/tasks/therapeutic_rediscovery/humab/tmp_manual/Alemtuzumab.fa
Anrukinzumab 0.88 0.809 ../../data/tasks/therapeutic_rediscovery/humab/tmp_manual/Anrukinzumab.fa
Atezolizumab 0.975 0.941 ../../data/tasks/therapeutic_rediscovery/humab/tmp_manual/Atezolizumab.fa
Bapineuzumab 0.97 0.6579999999999999 ../../data/tasks/therapeutic_rediscovery/humab/tmp_manual/Bapineuzumab.fa
Bemarituzumab 0.745 0.725 ../../data/tasks/therapeutic_rediscovery/humab/tmp_manual/Bemarituzumab.fa
Benralizumab 0.61 0.981 ../../data/tasks/therapeutic_rediscovery/humab/tmp_manual/Benralizumab.fa
Bevacizumab 0.81 0.899 ../../data/tasks/therapeutic_rediscovery/humab/tmp_manual/Bevaci

In [27]:
!cat {out_dir}/tmp_manual/*.fa > {out_dir}/humab_manual.fa

In [28]:
!cd ../../; make data/tasks/therapeutic_rediscovery/humab/humab_manual_oasis.xlsx 

source $(conda info --base)/bin/activate biophi && export APP_ENV=local  && biophi oasis \
        data/tasks/therapeutic_rediscovery/humab/humab_manual.fa \
        --output data/tasks/therapeutic_rediscovery/humab/humab_manual_oasis.xlsx \
        --oas-db sqlite:///../biophi/work/oas_clean_human_subject_9mers_2019_11.db \
        --proteome-db sqlite:///../biophi/work/uniprot_human_proteome_9mers.db
      __     ____  _       ____  _     _      ___    _   ____  _
  /| /  \   | __ )(_) ___ |  _ \| |__ (_)    / _ \  / \ / ___|(_)___
 ( || [] )  |  _ \| |/ _ \| |_) | '_ \| |   | | | |/ _ \\___ \| / __|
  \_   _/   | |_) | | (_) |  __/| | | | |   | |_| / ___ \___| | \__ \
    | |     |____/|_|\___/|_|   |_| |_|_|    \___/_/   \_\___/|_|___/
    |_|                       version 1.0                  version 1.0
────────────────────────────────────────────────────────────────────────

Settings:
- OAS database: sqlite:///../biophi/work/oas_clean_human_subject_9mers_2019_11.db
- Proteome da

In [29]:
!cd ../../; make data/tasks/therapeutic_rediscovery/humab/humab_manual_T20.tsv

bin/humanness_t20_score.py data/tasks/therapeutic_rediscovery/humab/humab_manual.fa data/tasks/therapeutic_rediscovery/humab/humab_manual_T20.tsv
Note: The sequences will be processed through lakepharma T20 service! Sleeping for 10s, press Ctrl+C to cancel...
Processing...
304it [16:25,  3.24s/it]
Saved to: data/tasks/therapeutic_rediscovery/humab/humab_manual_T20.tsv


In [16]:
H_THRESHOLDS = {
    'Sapelizumab': 0.7900,
    'Satralizumab': 0.7900
}

In [ ]:
for name in parental_heavy.keys():
    outpath = os.path.join(out_dir, 'tmp_automatic', name+'.fa')
    jobid_path = os.path.join(out_dir, 'tmp_automatic', name+'.jobid')
    h_threshold = scores['Hu-mAb Score (H)'][name]
    l_threshold = scores['Hu-mAb Score (L)'][name]
    print(name, h_threshold, l_threshold, outpath)
    if name in H_THRESHOLDS:
        print('  ! Replacing heavy chain threshold, using', H_THRESHOLDS[name], 'instead of', h_threshold)
        h_threshold = H_THRESHOLDS[name]
    if os.path.exists(outpath):
        continue
    jobname = name+'_Rediscovery'
    if os.path.exists(jobid_path):
        with open(jobid_path, 'rt') as f:
            jobid = f.read().strip()
    else:
        jobid = submit_humanization(
            h_sequence=parental_heavy[name], 
            h_v_gene='', 
            h_threshold=h_threshold, 
            l_sequence=parental_light[name], 
            l_v_gene='', 
            l_threshold=l_threshold, 
            jobname=jobname
        )
        with open(jobid_path, 'wt') as f:
            f.write(f'{jobid}\n')
    
    print('  Waiting for job:', 'http://opig.stats.ox.ac.uk/webapps/newsabdab/sabpred/humab_results/'+jobid)
    try:
        heavy_seq, light_seq = get_humanization_results(jobid, jobname=jobname)
        with open(outpath, 'wt') as f:
            f.write(f'>{name} VH (threshold {h_threshold})\n')
            f.write(f'{heavy_seq}\n')
            f.write(f'>{name} VL (threshold {l_threshold})\n')
            f.write(f'{light_seq}\n')
    except Exception as e:
        print('  FAILED', str(e))


Abituzumab 0.43 0.619 ../../data/tasks/therapeutic_rediscovery/humab_automatic/tmp/Abituzumab.fa
Afutuzumab 0.83 0.562 ../../data/tasks/therapeutic_rediscovery/humab_automatic/tmp/Afutuzumab.fa
Alacizumab 0.935 0.6859999999999999 ../../data/tasks/therapeutic_rediscovery/humab_automatic/tmp/Alacizumab.fa
Alemtuzumab 0.835 0.8420000000000001 ../../data/tasks/therapeutic_rediscovery/humab_automatic/tmp/Alemtuzumab.fa
Anrukinzumab 0.88 0.809 ../../data/tasks/therapeutic_rediscovery/humab_automatic/tmp/Anrukinzumab.fa
Atezolizumab 0.975 0.941 ../../data/tasks/therapeutic_rediscovery/humab_automatic/tmp/Atezolizumab.fa
Bapineuzumab 0.97 0.6579999999999999 ../../data/tasks/therapeutic_rediscovery/humab_automatic/tmp/Bapineuzumab.fa
Bemarituzumab 0.745 0.725 ../../data/tasks/therapeutic_rediscovery/humab_automatic/tmp/Bemarituzumab.fa
Benralizumab 0.61 0.981 ../../data/tasks/therapeutic_rediscovery/humab_automatic/tmp/Benralizumab.fa
Bevacizumab 0.81 0.899 ../../data/tasks/therapeutic_rediscov

In [36]:
!cat {out_dir}/tmp_automatic/*.fa > {out_dir}/humab_automatic.fa

In [41]:
!cd ../../; make data/tasks/therapeutic_rediscovery/humab/humab_automatic_oasis.xlsx 

source $(conda info --base)/bin/activate biophi && export APP_ENV=local  && biophi oasis \
        data/tasks/therapeutic_rediscovery/humab_automatic/humab.fa \
        --output data/tasks/therapeutic_rediscovery/humab_automatic/humab_oasis.xlsx \
        --oas-db sqlite:///../biophi/work/oas_clean_human_subject_9mers_2019_11.db \
        --proteome-db sqlite:///../biophi/work/uniprot_human_proteome_9mers.db
      __     ____  _       ____  _     _      ___    _   ____  _
  /| /  \   | __ )(_) ___ |  _ \| |__ (_)    / _ \  / \ / ___|(_)___
 ( || [] )  |  _ \| |/ _ \| |_) | '_ \| |   | | | |/ _ \\___ \| / __|
  \_   _/   | |_) | | (_) |  __/| | | | |   | |_| / ___ \___| | \__ \
    | |     |____/|_|\___/|_|   |_| |_|_|    \___/_/   \_\___/|_|___/
    |_|                       version 1.0                  version 1.0
────────────────────────────────────────────────────────────────────────

Settings:
- OAS database: sqlite:///../biophi/work/oas_clean_human_subject_9mers_2019_11.db
- Prote

In [43]:
!cd ../../; make data/tasks/therapeutic_rediscovery/humab/humab_automatic_T20.tsv

bin/humanness_t20_score.py data/tasks/therapeutic_rediscovery/humab_automatic/humab.fa data/tasks/therapeutic_rediscovery/humab_automatic/humab_T20.tsv
Note: The sequences will be processed through lakepharma T20 service! Sleeping for 10s, press Ctrl+C to cancel...
Processing...
304it [17:51,  3.53s/it]
Saved to: data/tasks/therapeutic_rediscovery/humab_automatic/humab_T20.tsv


## Germlines used in automatic germline selection

In [15]:
oasis_auto = pd.read_excel(os.path.join(TASK_DIR, 'humab/humab_automatic_oasis.xlsx'), index_col=0)
oasis_auto.head()

,Threshold,OASis Percentile,OASis Identity,Heavy V Germline,Heavy J Germline,Heavy OASis Percentile,Heavy OASis Identity,Heavy Non-human peptides,Light V Germline,Light J Germline,Light OASis Percentile,Light OASis Identity,Light Non-human peptides
Antibody,,,,,,,,,,,,,
Abituzumab,relaxed,0.093445,0.617225,IGHV1-46*01,IGHJ3*01,0.180182,0.654545,38,IGKV1-27*01,IGKJ2*01,0.061152,0.575758,42
Afutuzumab,relaxed,0.067465,0.534884,IGHV1-69*04,IGHJ4*01,0.070748,0.495495,56,IGKV2D-28*01,IGKJ2*01,0.061385,0.576923,44
Alacizumab,relaxed,0.126231,0.653846,IGHV3-NL1*01,IGHJ3*01,0.757183,0.871560,14,IGKV1/OR2-2*01,IGKJ2*01,0.014828,0.414141,58
Alemtuzumab,relaxed,0.092642,0.613208,IGHV3-71*01,IGHJ4*01,0.482743,0.769912,26,IGKV1D-39*01,IGKJ2*01,0.018000,0.434343,56
Anrukinzumab,relaxed,0.093944,0.619718,IGHV1-3*01,IGHJ4*01,0.201636,0.663636,37,IGKV7-3*01,IGKJ1*01,0.060563,0.572816,44


In [16]:
oasis_auto['Heavy V Germline'].apply(lambda s: s.split('-')[0].split('/')[0]).value_counts()

IGHV1    99
IGHV3    27
IGHV7    10
IGHV4     8
IGHV2     4
IGHV6     3
IGHV5     1
Name: Heavy V Germline, dtype: int64

In [17]:
oasis_auto['Light V Germline'].apply(lambda s: s.split('-')[0].split('/')[0]).value_counts()

IGKV1     37
IGKV1D    27
IGKV2     23
IGKV7     22
IGKV4     14
IGKV3     10
IGKV6D     6
IGKV2D     5
IGKV6      5
IGLV7      1
IGKV3D     1
IGKV5      1
Name: Light V Germline, dtype: int64

In [18]:
oasis_auto['Heavy V Germline'].apply(lambda s: s.split('*')[0]).value_counts()

IGHV1-2         35
IGHV1-46        27
IGHV1-69         9
IGHV3-30         8
IGHV7-4-1        8
IGHV1-18         8
IGHV1-3          7
IGHV1/OR15-1     7
IGHV4-59         4
IGHV1-8          4
IGHV3-74         3
IGHV3-71         3
IGHV6-1          3
IGHV1/OR15-5     2
IGHV3-23         2
IGHV3-11         2
IGHV7-81         2
IGHV3-66         2
IGHV4-38-2       2
IGHV2-70         2
IGHV3-69-1       2
IGHV2-5          2
IGHV5-51         1
IGHV3-NL1        1
IGHV3-33         1
IGHV3-30-5       1
IGHV4-30-4       1
IGHV4-34         1
IGHV3-48         1
IGHV3-21         1
Name: Heavy V Germline, dtype: int64

In [19]:
oasis_auto['Light V Germline'].apply(lambda s: s.split('*')[0]).value_counts()

IGKV7-3         22
IGKV4-1         14
IGKV2-30        14
IGKV1D-39       12
IGKV1D-33       12
IGKV1-12        10
IGKV3-11         7
IGKV1-8          6
IGKV2-29         6
IGKV6-21         5
IGKV1/OR10-1     5
IGKV2D-28        5
IGKV6D-21        4
IGKV1-NL1        4
IGKV1-27         4
IGKV3-7          3
IGKV1-9          2
IGKV1D-16        2
IGKV1-39         2
IGKV2-28         2
IGKV6D-41        2
IGKV1-33         1
IGKV1D-13        1
IGLV7-46         1
IGKV2-40         1
IGKV1-16         1
IGKV1-17         1
IGKV5-2          1
IGKV3D-7         1
IGKV1/OR2-2      1
Name: Light V Germline, dtype: int64